In [ ]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
from estival.sampling import tools as esamp
from tbdynamics.calib_utils import plot_spaghetti, plot_output_ranges
from tbdynamics.inputs import load_targets
import arviz as az
from tbdynamics.calib_utils import get_bcm

In [ ]:
OUT_PATH = Path.cwd() / 'runs/r1605'

In [ ]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
spaghetti = pd.read_hdf(OUT_PATH / 'results.hdf', 'spaghetti')
quantile_outputs = esamp.quantiles_for_results(spaghetti, quantiles)
targets = load_targets()

In [ ]:
plot_spaghetti(spaghetti, ['total_population','notification'], 2, targets)

In [ ]:
plot_spaghetti(spaghetti, ['prevalence_pulmonary','incidence'], 2, targets)

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['total_population','notification'], quantiles, 1, 1950, 2025)

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['prevalence_pulmonary','incidence', 'percentage_latent'], quantiles, 1, 2000, 2025)

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['cdr'], quantiles, 1, 2000, 2025)

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['mortality_raw'], quantiles, 1, 2010, 2025)

In [ ]:
idata = az.from_netcdf(OUT_PATH / 'calib_full_out.nc')

In [ ]:
idata = idata.sel(draw=slice(5000, None))

In [ ]:
az.plot_trace(idata, figsize=(16,3.1*len(idata.posterior)))

In [ ]:
az.summary(idata)

In [ ]:
params = {
    # "contact_rate": 0.007255277552812773,
    "start_population_size": 2100000.0,
    # "rr_infection_latent": 0.4435087078206438,
    # "rr_infection_recovered": 0.3726339126072788,
    # "progression_multiplier": 1.71734470040371,
    "seed_time": 1805.0,
    "seed_num": 10.0,
    "seed_duration": 20.0,
    # "smear_positive_death_rate": 0.3789083132417181,
    # "smear_negative_death_rate": 0.023640460692223537,
    # "smear_positive_self_recovery": 0.2064856493052769,
    # "smear_negative_self_recovery": 0.11108908197816095,
    # "screening_scaleup_shape": 0.07019338853789103,
    # "screening_inflection_time": 1994.2958903597957,
    # "screening_end_asymp": 0.5881461121229823,
    # "detection_reduction": 0.7542772690107872,
}
bcm = get_bcm(params)

In [ ]:
idata_extract = az.extract(idata, num_samples=200)

In [ ]:
spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm)

In [ ]:
spaghetti_res.to

In [ ]:
qoutputs = esamp.quantiles_for_results(spaghetti_res, quantiles)
targets = load_targets()

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['total_population','notification'], quantiles, 1, 1950, 2025)

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['prevalence_pulmonary','incidence', 'percentage_latent'], quantiles, 1, 2000, 2025)

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['cdr'], quantiles, 1, 2000, 2025)